In [ ]:
import pandas as pd
import time
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
from utils.utils import *

In [ ]:
x = points_on_line(np.array([0,0,0]),np.array([0,1,0]),1000)

In [9]:
def feet_inline_grader(hold_time,dist_heels_l,dist_heels_r,dist_indices_l,dist_indices_r,dist_heel_index):
    distance_band_one = 0.3
    distance_band_two = 0.5
    distance_band_three = 0.8
    if dist_heels_l < distance_band_one and dist_heels_r < distance_band_one and dist_indices_l < distance_band_one and dist_indices_r < distance_band_one and  dist_heel_index < distance_band_one and (hold_time > 30):
        return 4
    if dist_heels_l < distance_band_one and dist_heels_r < distance_band_one and dist_indices_l < distance_band_one and dist_indices_r < distance_band_one and  (distance_band_one < dist_heel_index < distance_band_two) and (hold_time > 30):
        return 3
    if dist_heels_l < distance_band_one and dist_heels_r < distance_band_one and dist_indices_l < distance_band_one and dist_indices_r < distance_band_one and  (distance_band_two < dist_heel_index < distance_band_three) and (hold_time > 30):
        return 2
    if dist_heels_l < distance_band_one and dist_heels_r < distance_band_one and dist_indices_l < distance_band_one and dist_indices_r < distance_band_one and  (distance_band_two < dist_heel_index < distance_band_three) and (hold_time > 15):
        return 1
    else:
        return 0
    

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
distance_band = 0.3
cap = cv2.VideoCapture('./short_form.mp4')
captured_video_fps = cap.get(cv2.CAP_PROP_FPS)
#####FLAGS#####
first_time = False
was_overlooked = False
confirm_addition = False
success = False
###############
#####GLOBALS#####
left_heel_original_coordinate = None
right_heel_original_coordinate = None
left_foot_index_original_coordinate = None
right_foot_index_original_coordinate = None
hold_time_start = None
hold_time_end = None
overlooked_array = []
time_of_last_addition = 0
tries = 0
#################
frame_idx = 0
start_time = cv2.getTickCount()
############################
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    frame_idx+=1
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
      ######################################################
      #Extract Joint coordinates
      my_landmarks = results.pose_landmarks.landmark
      key = cv2.waitKey(33) & 0xFF
      right_heel = np.array([my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x , my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y, my_landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].z])
      left_heel = np.array([my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x , my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y, my_landmarks[mp_pose.PoseLandmark.LEFT_HEEL].z])
      left_ankle = np.array([my_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x , my_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y, my_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].z])
      right_ankle = np.array([my_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x , my_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y, my_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].z])
      left_foot_index = np.array([my_landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x , my_landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y, my_landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z])
      right_foot_index = np.array([my_landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x , my_landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y, my_landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z])

      #############################
      end_time = cv2.getTickCount()   
      total_time = (end_time - start_time) / cv2.getTickFrequency()
      # print("The fps is: ",frame_idx/total_time)
      multiply_factor = 20/(frame_idx/total_time)
      real_time = total_time/multiply_factor

      #################################################################
      if key == ord('s'):
        print('evaluation started.')
        joint_coordinates = get_coordinates(my_landmarks)
        
        if not first_time:
          left_heel_original_coordinate = left_heel
          right_heel_original_coordinate = right_heel
          left_foot_index_original_coordinate = left_foot_index
          right_foot_index_original_coordinate = right_foot_index
          hold_time_start = real_time
          if left_heel[0] < right_heel[0]:
            #left foot is placed before the right one.
            distance_between_index_and_heel = distance(left_foot_index,right_heel)
            if distance_between_index_and_heel < distance_band:
              print("foot placed infront of each other(no space.)")
          if right_heel[0] < left_heel[0]:
            #right foot is placed before the left one.
            distance_between_index_and_heel = distance(right_foot_index,left_heel)
            if distance_between_index_and_heel < distance_band:
              print("foot placed infront of each other(no space.)")
          first_time = True
          # success  =True
        #########RUNS AFTER FIRST TIME############
      if first_time:
        if left_heel[0] < right_heel[0]:
          distance_between_index_and_heel = distance(left_foot_index,right_heel)
          distance_between_heels_left = distance(left_heel,left_heel_original_coordinate)
          distance_between_heels_right = distance(right_heel,right_heel_original_coordinate)          

          distance_between_indices_left = distance(left_foot_index,left_foot_index_original_coordinate)
          distance_between_indices_right = distance(right_foot_index,right_foot_index_original_coordinate)

          if distance_between_heels_left < distance_band and distance_between_heels_right < distance_band and distance_between_indices_left < distance_band and distance_between_indices_right < distance_band and distance_between_index_and_heel < distance_band:
            print('right foot placed infront of the left(No space.)')
          else:
            overlooked_array.append(1)
            time_of_last_addition = time.time()
            confirm_addition = True
        if right_heel[0] < left_heel[0]:
          distance_between_index_and_heel = distance(right_foot_index,left_heel)
          distance_between_heels_left = distance(left_heel,left_heel_original_coordinate)
          distance_between_heels_right = distance(right_heel,right_heel_original_coordinate)          

          distance_between_indices_left = distance(left_foot_index,left_foot_index_original_coordinate)
          distance_between_indices_right = distance(right_foot_index,right_foot_index_original_coordinate)

          if distance_between_heels_left < distance_band and distance_between_heels_right < distance_band and distance_between_indices_left < distance_band and distance_between_indices_right < distance_band and distance_between_index_and_heel < distance_band:
            print('left foot placed infront of the right(No space.)')
          else:
            overlooked_array.append(1)
            time_of_last_addition = time.time()
            confirm_addition = True
          ##########################################
        
        if len(overlooked_array) > 30:
          final_hold_time = real_time - hold_time_start
          res = feet_inline_grader(hold_time,distance_between_heels_left,
                                   distance_between_heels_right,
                                   distance_between_indices_left,
                                   distance_between_indices_right,
                                   distance_between_index_and_heel)
          if res == 4:
            print("The exersize Was completed with a Score of: ",res)
            break
          if res == 3:
            print("The exersize Was completed with a Score of: ",res)
            break
          if res == 2:
            print("The exersize Was completed with a Score of: ",res)
            break
          if res == 1:
            print("The exersize Was completed with a Score of: ",res)
            break
          if res == 0:
            print("The exersize Was completed with a Score of: ",res)
            break
          hold_time_start = real_time
          tries +=1
        now = time.time()
        if confirm_addition:
          if now - time_of_last_addition >= 2:
            overlooked_array.clear()
        if real_time - hold_time_start > 30:
          print("The exersize Was completed with a Score of: ",4)
          break
        if tries >=3:
          hold_time = real_time - hold_time_start
          res = feet_inline_grader(hold_time,distance_between_heels_left,
                                   distance_between_heels_right,
                                   distance_between_indices_left,
                                   distance_between_indices_right,
                                   distance_between_index_and_heel)
          print("The exersize completed with a grade of: ",res)
          break
      cv2.rectangle(image, (0, 0), (640,60), (255, 0, 0), -1)
      cv2.putText(image,str(real_time-hold_time_start), (50, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
      #################################################################
    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    flipped_frame = cv2.flip(image, 0)

    cv2.imshow('MediaPipe Holistic',image)
    is_right_leg_lifted = False
    is_left_leg_lifted = False
    if cv2.waitKey(33) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()